In [ ]:
!date

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.conv_learner import *
from planet import *
import operator
import os

In [ ]:
project_name = 'planet-understanding-the-amazon-from-space'

In [ ]:
PATH='/home/paperspace/data/planet/'
assert os.path.exists(PATH)

In [ ]:
arch=resnet50
size=64
bs=64
metrics=[f2]

In [ ]:
def get_data(sz):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.05)
    return ImageClassifierData.from_csv(PATH, 'train-jpg', label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name='test-jpg')

In [ ]:
label_csv = f'{PATH}train_v2.csv'

In [ ]:
no_of_rows = len(list(open(label_csv)))-1

In [ ]:
val_idxs = get_cv_idxs(no_of_rows)

In [ ]:
print(val_idxs.shape)
val_idxs

In [ ]:
label_df = pd.read_csv(label_csv)

In [ ]:
label_df.head()

In [ ]:
label_df.pivot_table(index='tags', aggfunc=len).sort_values('image_name', ascending=False)[:10]

In [ ]:
data = get_data(size)

In [ ]:
x, y = next(iter(data.val_dl))

In [ ]:
y

In [ ]:
list(zip(data.classes, y[0]))

In [ ]:
plt.imshow(data.val_ds.denorm( to_np(x) )[0] )

In [ ]:
plt.imshow(data.val_ds.denorm( to_np(x) )[0] * 1.3 ) # TIP : Multiply by a value if the image is too hazy

In [ ]:
data = get_data(size)

In [ ]:
data = data.resize(int(size*1.3), 'tmp')

In [ ]:
plt.imshow(data.val_ds.denorm( to_np(x) )[0] )

## Train model

In [ ]:
learn = ConvLearner.pretrained(arch, data, metrics=metrics, precompute=False)

In [ ]:
lrf = learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
?? metrics

In [ ]:
learning_rate = 0.1

In [ ]:
%time learn.fit(learning_rate, 3, cycle_len=1, metrics=metrics)

In [ ]:
learn.sched.plot_loss()

##### As sat images are not part of imagnet, we divide by smaller numbers as we want to train the entire network from scratch. Well kind of, we initially use the weights from imagnet model as something is better than nothing and also, imagnet initial layers actually detect basic structures like edges etc.

In [ ]:
learning_rates = np.array([learning_rate/9, learning_rate/3, learning_rate]) 

In [ ]:
learn.unfreeze()
learn.bn_freeze(True)

In [ ]:
%time learn.fit(learning_rates, 3, cycle_len=1, cycle_mult=2, metrics=metrics)

In [ ]:
%time learn.save(f'{project_name}_last_layer_{size}')

In [ ]:
learn.sched.plot_loss()

### Increase pic size

In [ ]:
size = 128

In [ ]:
learn.set_data(get_data(size))
learn.freeze()
%time learn.fit(learning_rate, 3, cycle_len=1, metrics=metrics)
learn.sched.plot_loss()

In [ ]:
learn.unfreeze()
learn.bn_freeze(True)
%time learn.fit(learning_rates, 3, cycle_len=1, cycle_mult=2, metrics=metrics)
learn.sched.plot_loss()
learn.save(f'{project_name}_last_layer_{size}')

#### 256 size

In [ ]:
size = 256

In [ ]:
learn.set_data(get_data(size))
learn.freeze()
%time learn.fit(learning_rate, 3, cycle_len=1, metrics=metrics)
learn.sched.plot_loss()

In [ ]:
learn.unfreeze()
learn.bn_freeze(True)
%time learn.fit(learning_rates, 3, cycle_len=1, cycle_mult=2, metrics=metrics)
learn.sched.plot_loss()
learn.save(f'{project_name}_last_layer_{size}')

In [ ]:
learn = learn.load((f'{project_name}_last_layer_{size}'))

In [ ]:
multi_preds, y = learn.TTA()
preds = np.mean(multi_preds, 0)

In [ ]:
print(multi_preds.shape)
print(preds.shape)
print(y.shape)

In [ ]:
preds[0]

In [ ]:
y[0]

In [ ]:
# f2(preds,y)

In [ ]:
#tta = learn.TTA()

In [ ]:
# f2(*tta)

In [ ]:
test_multi_preds, test_y = learn.TTA(is_test=True) # doesn't return test_y for test set as the label is to be predicted!
test_preds = np.mean(test_multi_preds, 0)

In [ ]:
print(test_multi_preds.shape)
print(test_preds.shape)
print(test_y.shape)

In [ ]:
test_y.max()

In [ ]:
test_preds[0]

In [ ]:
test_preds[240]

In [ ]:
test_probs = np.round(test_preds)

In [ ]:
test_probs.shape

In [ ]:
test_probs[0]

In [ ]:
print(test_probs[240]) # all zeros?

In [ ]:
f2(test_preds, test_probs)

In [ ]:
#def get_test_classes(probs, classes):
#    prob_idxs = np.argwhere(probs==1) # Indexs of 1's in every row of probs ([0 1 0 1 0 1 0])
#    prob_idxs = prob_idxs.flatten().tolist() # Convert the o/p to a list after flatting it as required by itemgetter
    #return operator.itemgetter(*prob_idxs)(classes) 

In [ ]:
def get_test_classes(probs, classes):
    prob_idxs = np.argwhere(probs==1) # Indexs of 1's in every row of probs ([0 1 0 1 0 1 0])
    prob_idxs = prob_idxs.flatten().tolist() # Convert the o/p to a list after flatting it as required by itemgetter

    all_probs_zero = 0
    if not prob_idxs:
        all_probs_zero =+ 1
        prob_idxs = [10]  # default to haze

    if all_probs_zero != 0: print("All 0 probs : {}".format(all_probs_zero))
    #return operator.itemgetter(*(np.argwhere(probs==1)).flatten().tolist())(classes)    

    classes = operator.itemgetter(*prob_idxs)(classes)
    # If there is just one class, then the itemgetter returns a string instead of a tuple
    # Convert to tuple. Else, the join will put a space in between every letter in the aingle word.
    # E.g.: c l o u d y instead of cludy
    if type(classes) == str: classes = (classes,) 
        
    return classes

In [ ]:
a = get_test_classes(test_probs[4], learn.data.classes)
print(type(a))
print(len(a))
a

In [ ]:
#operator.itemgetter(a)(learn.data.classes) 
#test_classes = [' '.join( get_test_classes(probs, learn.data.classes) ) for probs in test_probs]
#[print( ' '.join( operator.itemgetter(*(np.argwhere(a==1)).flatten().tolist())(learn.data.classes) ) ) for a in test_probs[230:250]]
#test_classes = [' '.join( operator.itemgetter(*(np.argwhere(a==1)).flatten().tolist())(learn.data.classes) )  for a in test_probs]

In [ ]:

test_classes = [ ' '.join( get_test_classes(probs, learn.data.classes) ) for probs in test_probs]


In [ ]:
test_classes[:5]
#print(len(test_classes))

In [ ]:
fn = PATH + get_data(224).test_ds.fnames[240]
fn

In [ ]:
fnames = [f[9:-4] for f in learn.data.test_ds.fnames]
len(fnames)

In [ ]:
fnames[240]

In [ ]:
subm = np.stack([fnames, test_classes], axis=1)
subm[:5]

In [ ]:
test_df = pd.DataFrame(subm)

In [ ]:
test_df.head()

In [ ]:
#test_df.insert(0, 'image_name', fnames )

# Submit to kaggle

In [ ]:
!pwd

In [ ]:
now = datetime.datetime.now()
submission_file_name = f'{project_name}-{str(now)}.csv'
# changed the decimal format from .5 to .10 and NO clip and moved into top 8% on kaggle (from 10%)
np.savetxt(submission_file_name, subm, fmt='%s,%s', header='image_name,tags', comments='')

In [ ]:
from IPython.display import FileLink
FileLink(submission_file_name)

In [ ]:
!date